In [1]:
1+1

2

In [2]:
!pip install arxiv


In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [4]:
!pip install wikipedia

In [5]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [6]:
wiki.name

'wikipedia'

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:

## load,chunk and index the content of the html page

loader=WebBaseLoader("https://python.langchain.com/docs/modules/tools/")


In [69]:
docs=loader.load()
docs

[Document(page_content='\n\n\n\n\nTools | 🦜️🔗 LangChain\n\n\n\n\n\n\n\nSkip to main contentComponentsIntegrationsGuidesAPI ReferenceMorePeopleVersioningContributingTemplatesCookbooksTutorialsYouTube🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchModel I/OPromptsChat modelsLLMsOutput parsersRetrievalDocument loadersText splittersEmbedding modelsVector storesRetrieversIndexingCompositionToolsToolkitsToolsDefining Custom ToolsTools as OpenAI FunctionsAgentsChainsMoreComponentsCompositionToolsOn this pageToolsTools are interfaces that an agent, chain, or LLM can use to interact\nwith the world. They combine a few things:The name of the toolA description of what the tool isJSON schema of what the inputs to the tool areThe function to callWhether the result of a tool should be returned directly to the userIt is useful to have all this information because this information can\nbe used to build action-taking systems! The name, description, a

In [10]:
docs

[Document(page_content='\n\n\n\n\nTools | 🦜️🔗 LangChain\n\n\n\n\n\n\n\nSkip to main contentComponentsIntegrationsGuidesAPI ReferenceMorePeopleVersioningContributingTemplatesCookbooksTutorialsYouTube🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchModel I/OPromptsChat modelsLLMsOutput parsersRetrievalDocument loadersText splittersEmbedding modelsVector storesRetrieversIndexingCompositionToolsToolkitsToolsDefining Custom ToolsTools as OpenAI FunctionsAgentsChainsMoreComponentsCompositionToolsOn this pageToolsTools are interfaces that an agent, chain, or LLM can use to interact\nwith the world. They combine a few things:The name of the toolA description of what the tool isJSON schema of what the inputs to the tool areThe function to callWhether the result of a tool should be returned directly to the userIt is useful to have all this information because this information can\nbe used to build action-taking systems! The name, description, a

In [70]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [71]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

In [109]:
from dotenv import load_dotenv
#import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()
import os
os.getenv("GOOGLE_API_KEY")

#genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

#llm = genai.GenerativeModel('gemini-pro')
llm = ChatGoogleGenerativeAI(model="gemini-pro",convert_system_message_to_human=True, transport='rest')


In [110]:
llm

ChatGoogleGenerativeAI(model='gemini-pro', client=genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
), convert_system_message_to_human=True)

In [35]:
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)
# llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001E78ABEED70>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001E78AC10250>, model_name='gpt-3.5-turbo-1106', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

In [75]:
## FAISS Vector Database
from langchain_community.vectorstores import FAISS
vector_db = FAISS.from_documents(documents[:15], OllamaEmbeddings())

In [111]:
retriever=vector_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E78C1E5030>)

In [112]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langchain_tools","give difference between langchain tools and langchain chains")

In [113]:
retriever_tool.name

'langchain_tools'

In [114]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [115]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [116]:
arxiv.name

'arxiv'

In [117]:
arxiv

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>))

In [118]:
tools=[wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\arjun.sreekumar\\AppData\\Local\\anaconda3\\envs\\rag_pipeline\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langchain_tools', description='give difference between langchain tools and langchain chains', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001E7FD46AB90>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_comm

In [119]:
pip install langchainhub

Note: you may need to restart the kernel to use updated packages.


In [120]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [121]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history")

In [122]:
##Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)


In [123]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, mess

In [124]:
llm

ChatGoogleGenerativeAI(model='gemini-pro', client=genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
), convert_system_message_to_human=True)

In [125]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools,verbose=True)

In [126]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [127]:
result = llm.invoke("hello")
print(result.content)

c:\Users\arjun.sreekumar\AppData\Local\anaconda3\envs\rag_pipeline\lib\site-packages\langchain_google_genai\chat_models.py:308: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Hi there! How can I help you today?


In [128]:
agent_executor.invoke({"input":"tell me about langserve"})



> Entering new AgentExecutor chain...


c:\Users\arjun.sreekumar\AppData\Local\anaconda3\envs\rag_pipeline\lib\site-packages\langchain_google_genai\chat_models.py:308: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


ValueError: Unknown field for FunctionDeclaration: type

In [29]:
pip install -U langchain-cli

Note: you may need to restart the kernel to use updated packages.


In [34]:
from gemini_functions_agent import agent_executor

agent_executor.invoke({"input":"tell me about langserve"})

#add_routes(app, gemini_functions_agent_chain, path="/openai-functions-agent")

ModuleNotFoundError: No module named 'gemini_functions_agent'